Для задания были взяты 2 слова: собака и платье. Для каждого слова было взято 750 контекстов из НКРЯ, которые я заранее очистила от ссылок на авторов, и заменила эти слова на ХХХ.

In [170]:
import numpy as np
import pandas as pd
import gensim
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.svm import LinearSVC


from stop_words import get_stop_words
stop_words_421 = get_stop_words('russian')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



In [171]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

#### кодирование текстов: для собаки 1, для платья 0

In [172]:
dog = open('dog закодировано.txt', encoding='UTF-8').read().split('\n')
dress = open('dress закодировано.txt', encoding='UTF-8').read().split('\n')

In [173]:
doggy = [text for text in dog]
dresss = [text for text in dress]

In [174]:
df = pd.DataFrame(doggy[1:], columns=['text'])

In [175]:
df['class'] = 1

In [176]:
df_2 = pd.DataFrame(dress[1:], columns=['text'])

In [177]:
df_2['class'] = 0

In [178]:
df = df.append(df_2, ignore_index=True)

In [179]:
df.head()

,text,class
0,Там жить невозможно. Даже XXX живут лучше. ...,1
1,"И обиженные, точнее, несправедливо обиженные ...",1
2,"И они― люди, а не стая бездомных XXX.",1
3,"XXX сеттер подбежала и тут же съела то, что е...",1
4,"XXX Найда любит до самоотречения, смотрит с к...",1


### Токенизация и лемматизация и удаление пустых строк

In [180]:
df['text'].replace('', np.nan, inplace=True)
df.dropna(subset=['text'], inplace=True)

In [181]:
sents = []
for sent in df.text:
    arr = []
    words = word_tokenize(sent)
    for word in words:
        lemma = morph.parse(word)[0].normal_form
        if not '\n' in lemma and lemma not in stop_words_421:
            if any([x.isalpha() for x in lemma]):
                arr.append(lemma)
    if arr:
        sents.append(arr)
    else:
        print(":{}:".format(sent))

### TF IDF Vectorizing

In [182]:
vectorizer = TfidfVectorizer()

sents_tfidf = [' '.join([x for x in sent if x != 'XXX' and x not in stop_words_421]) for sent in sents]
print(sents_tfidf[2])

они― стая бездомный xxx


In [183]:
X_train, X_test_raw, y_train, y_test = train_test_split(sents_tfidf, df['class'], test_size=0.25, random_state=1337)

In [184]:
tfidf_svc = LinearSVC(random_state=1337)
tfidf_rf = RandomForestClassifier(random_state=1337)
tfidf_gbc = GradientBoostingClassifier(n_estimators=250, random_state=1337)

In [185]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test_raw)

In [186]:
for classif in [tfidf_svc, tfidf_rf, tfidf_gbc]:
    classif.fit(X_train, y_train)
    y_pred = classif.predict(X_test)
    score = f1_score(y_test, y_pred)
    print(score)

0.8163265306122449
0.7612293144208039
0.7718446601941747


### Word 2 Vec

In [187]:
df_vectors = []

model = gensim.models.Word2Vec(sents, size=700, window=3, min_count=5, workers=4)

for sent in sents:
    vec = []
    for word in sent:
        try:
            vec.append(model[word])
        except:
            pass
    
    try:
        df_vectors.append(sum(vec)/len(vec))
    except:
        print(sent)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [188]:
df.shape 

(1496, 2)

In [189]:
print(len(df_vecs))

1496


In [190]:
X_train, X_test, y_train, y_test = train_test_split(df_vectors, df['class'], test_size=0.25, random_state=1337)

In [191]:
w2v_svc = LinearSVC(random_state=1337)
w2v_rf = RandomForestClassifier(random_state=1337)
w2v_gbc = GradientBoostingClassifier(n_estimators=250, random_state=1337)

In [192]:
for classif in [w2v_svc, w2v_rf, w2v_gbc]:
    classif.fit(X_train, y_train)
    y_pred = classif.predict(X_test)
    score = f1_score(y_test, y_pred)
    print(score)
    

0.5225225225225225
0.4628571428571428
0.6170212765957447


### Уменьшение размерности для TF IDF

Я решила уменьшить размерность, оставив только наиболее важные слова в текстах, т.е. которые появлялись бы в корпусе 2 раза и больше.

In [193]:
vectorizer_2 = TfidfVectorizer(min_df=2)

sents_tfidf_2 = [' '.join([x for x in sent if x != 'XXX' and x not in stop_words_421]) for sent in sents]
print(sents_tfidf[2])

они― стая бездомный xxx


In [194]:
X_train, X_test_raw, y_train, y_test = train_test_split(sents_tfidf_2, df['class'], test_size=0.25, random_state=1337)

In [195]:
tfidf_svc_2 = LinearSVC(random_state=1337)
tfidf_rf_2 = RandomForestClassifier(random_state=1337)
tfidf_gbc_2 = GradientBoostingClassifier(n_estimators=250, random_state=1337)

In [196]:
X_train = vectorizer_2.fit_transform(X_train)
X_test = vectorizer_2.transform(X_test_raw)

In [197]:
for classif in [tfidf_svc_2, tfidf_rf_2, tfidf_gbc_2]:
    classif.fit(X_train, y_train)
    y_pred = classif.predict(X_test)
    score = f1_score(y_test, y_pred)
    print(score)

0.7989556135770234
0.7427184466019418
0.7648456057007127


### Уменьшение размерности Word 2 Vec с 700 до 100

In [198]:
df_vectors_2 = []

model_sto = gensim.models.Word2Vec(sents, size=100, window=2, min_count=5, workers=4)

for sent in sents:
    vec = []
    for word in sent:
        try:
            vec.append(model_sto[word])
        except:
            pass
    
    try:
        df_vectors_2.append(sum(vec)/len(vec))
    except:
        print(sent)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [199]:
X_train, X_test, y_train, y_test = train_test_split(df_vectors_2, df['class'], test_size=0.25, random_state=1337)

In [200]:
sto_w2v_svc = LinearSVC(random_state=1337)
sto_w2v_rf = RandomForestClassifier(random_state=1337)
sto_w2v_gbc = GradientBoostingClassifier(n_estimators=250, random_state=1337)

In [201]:
for classif in [sto_w2v_svc, sto_w2v_rf, sto_w2v_gbc]:
    classif.fit(X_train, y_train)
    y_pred = classif.predict(X_test)
    score = f1_score(y_test, y_pred)
    print(score)

0.5060240963855422
0.47305389221556887
0.6120218579234973


## Сравнение показателей

* tfidf_svc, 0.8163265306122449
* tfidf_rf, 0.7612293144208039
* tfidf_gbc, 0.7718446601941747

  
* w2v_svc, 0.5225225225225225
* w2v_rf, 0.4628571428571428
* w2v_gbc,0.6170212765957447



* tfidf_svc_2,  0.7989556135770234
* tfidf_rf_2, 0.7427184466019418
* tfidf_gbc_2, 0.7648456057007127


* sto_w2v_svc, 0.5060240963855422
* sto_w2v_rf, 0.47305389221556887
* sto_w2v_gbc, 0.6120218579234973



### Предсказание на своих примерах

In [202]:
ex_dog = ['Собака лучший друг человека', 'Петя любит гулять со своей собакой','У нас во дворе живет пять собак', 'Собаки обожают когда им чешешь живот', 'Мне надо накормить свою собаку'] 
ex_dress = ['Маша купила новое платье вчера', 'В магазине продаются очень красивые платья','Я надела новое платье на праздник', 'На красную дорожку актрисы выходят в дорогих платьях', 'У Кати коллекция дорогих и красивых платьев'] 

In [203]:
for sent in ex_dog:
    sent = ''.join(morph.parse(sent)[0].normal_form)
    vector = vectorizer.transform([sent])
    print(sent)
    print(('prediction:{}').format(tfidf_svc.predict(vector)))

собака лучший друг человек
prediction:[1]
петя любит гулять со своей собака
prediction:[1]
у нас во дворе живет пять собака
prediction:[1]
собаки обожают когда им чешешь живот
prediction:[0]
мне надо накормить свою собак
prediction:[1]


In [204]:
for sent1 in ex_dress:
    sent1 = ''.join(morph.parse(sent1)[0].normal_form)
    vector1 = vectorizer.transform([sent1])
    print(sent1)
    print(('prediction:{}').format(tfidf_svc.predict(vector1)))

маша купила новое платье вчера
prediction:[0]
в магазине продаются очень красивые платья
prediction:[1]
я надела новое платье на праздник
prediction:[0]
на красную дорожку актрисы выходят в дорогих платие
prediction:[1]
у кати коллекция дорогих и красивых платьев
prediction:[1]


Несмотря, на то, что этот метод имеет самый лучший показатель (0.8163265306122449) из всех, он тоже часто ошибается. Особенно в примерах с платьем.